In [ ]:
# Fonte de dados- Estadual
# Obtendo dados do diario Estadual

# #xemplos de links 

# https://ioes.dio.es.gov.br/portal/visualizacoes/pdf/5293/#/p:1/e:5293
# https://ioes.dio.es.gov.br/portal/visualizacoes/pdf/5288/#/p:2/e:5288
# https://ioes.dio.es.gov.br/portal/visualizacoes/pdf/5288/#/p:1/e:5288

import wget
import os

#wget.download('https://ioes.dio.es.gov.br/portal/edicoes/download/5293')
# diario_oficial_2020-12-08_completo.pdf

#wget.download('https://ioes.dio.es.gov.br/portal/edicoes/download/5288')
#wget.download('https://ioes.dio.es.gov.br/portal/edicoes/download/5284')

# ja feito
# 0 a 1000 (interrompido)
# 1000 a 1100 erro
# 1100 a 1200
# 5200 a 6600

aux = 0
for i in range(aux,8000):
    aux = aux + i
    i = i + 1
    try:
        wget.download('https://ioes.dio.es.gov.br/portal/edicoes/download/'+str(aux))
    except:
        print("excecao")
        continue

In [ ]:
# Exemplo de trecho de nomeacao no diario estadual, diferente dos anteriores

# NOMEAR, de acordo com o Art.12, inciso II, da Lei Complementar nº 46, de 31 de janeiro de 1994, EMANUEL MALTA ROSA, para
#exercer o cargo de provimento em comissão de Assessor Jurídico do Sistema Penal, Ref.

# Nem sempre existe o Ref. Por exemplo,
# NOMEAR, de acordo com o Artigo 74, § 2º, inciso I, alínea “b”, da Constituição Estadual, DOMINGOS AUGUSTO TAUFNER, para
# exercer o cargo de Conselheiro do Tribunal de Contas do Estado do Espírito Santo.

# Lendo o conteudo dos pdfs

from PyPDF2 import PdfFileReader
import re
import os
import csv

for filename in os.listdir("C:\\temp\\es_gov\\"):
    if filename.endswith(".pdf"): 
        #input = PdfFileReader("C:\\temp\\es_gov\\diario_oficial_2020-12-08_completo.pdf")
        input = PdfFileReader("C:\\temp\\es_gov\\diario_oficial_2021-02-03_completo.pdf")
        #input = PdfFileReader("C:\\temp\\es_gov\\"+filename)
        NumPages = input.getNumPages()

        String = "NOMEAR"
        StringFim = "Protocolo"

        for i in range(0, NumPages):
            aux = ""
          #  print('pagina ' , i)
            PageObj = input.getPage(i)
            Text = PageObj.extractText() 
            ResSearch1 = re.search(String, Text)
            ResSearch2 = re.search(StringFim, Text)
            if Text.partition(String)[2].partition(StringFim)[0] is not "":
                aux = Text.partition(String)[2].partition(StringFim)[0]
                try:
                    # x1 = re.search(expRegular,Text)
            
                    # Obtem o nome da pessoa, após reduzir para o trecho de nomeação
            
                   # expRegular = "\,\s{1,}[A-Z]{3,}\s[A-Z]{2,}\s[A-Z]{2,}\s*[A-Z]*\,"
                    expRegular = "\,\s{0,}[A-Z]{3,}\s[A-Z]{2,}\s[A-Z]{2,}(.*?)\,"
            
                    x1 = re.search(expRegular, aux, re.DOTALL) 
                
                    for i in range(0,len(x1.groups())):
                        print("G" + str(i) + " "+ x1.group(i))
                
                    print(filename)
                
                    # Obtem o cargo
                
                    #expRegularCargo1 = x1.group()+"\s{0,}para(.*?)o cargo(.*?)\s{0,}(.*?)\s{0,}Ref."
                    #expRegularCargo1 = x1.group()+"\s{0,}para\s{0,}(.*?)\s{0,}cargo\s{0,}(.*?)\s{0,}\."
                    #expRegularCargo1 = x1.group()+"\s{0,}para\s{0,}(.*?)\s{0,}cargo\s{0,}(.*?)\s{0,}(.*?)\s{0,}(.*?)\s{0,}\."
                    #expRegularCargo1 = x1.group()+"\s{0,}para(.*?)cargo(.*?)\."
                    expRegularCargo1 = x1.group(0)+"\s{0,}para\s{0,}(.*)\s{0,}cargo\s{0,}(.*)\s{0,}(.*)\s{0,}(.*)\s{0,}\."
                    
                    #cargo1=re.search(expRegularCargo1,aux, flags = (re.DOTALL))
                    #cargo1=re.search(expRegularCargo1,aux, flags = (re.DOTALL | re.MULTILINE))
                    #cargo1=re.search(expRegularCargo1,aux, flags = (re.MULTILINE))
                    cargo1=re.search(expRegularCargo1,aux)
                    
                    print("########## printando a string inteira do match")
                    print(cargo1)
                    print("########## group()")
                    print(cargo1.group())
                    print("#########")
                    
                    for i in range(0,len(cargo1.groups())):
                        print("G" + str(i) + " "+ cargo1.group(i))
                    
                    # Erro na leitura do pdf, está pulando alguns trechos
                    
                    
                    # grava arquivo csv
                    with open('C:\\temp\\es_gov\\'+filename+'_FINAL.csv', 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile, delimiter=';')
                        lista = [0,x1,cargo1.group(2),filename,"Estadual","ES"]
                        writer.writerow(lista)
                
                except Exception as e:
                    print(e)
                    continue

In [ ]:
################# Definitivo com a api correta

import fitz 
import re
import os
import csv

for filename in os.listdir("C:\\temp\\es_gov\\"):
    if filename.endswith(".pdf"): 
        input = doc = fitz.open("C:\\Temp\\es_gov\\"+filename)

        String = "NOMEAR"
        StringFim = "Protocolo"
        
        text_list=[] 
        for i in range(input.pageCount):  
            pg=input.loadPage(i) 
            Text=pg.getText() 
            #text_list.append(Text) 
            ResSearch1 = re.search(String, Text)
            ResSearch2 = re.search(StringFim, Text)
            if Text.partition(String)[2].partition(StringFim)[0] is not "":
                aux = Text.partition(String)[2].partition(StringFim)[0]
                try:
                    # Obtem o nome da pessoa, após reduzir para o trecho de nomeação
                    expRegular = "\,\s{0,}[A-Z]{3,}\s[A-Z]{2,}\s[A-Z]{2,}(.*?)\,"
                    
                    x1 = re.search(expRegular, aux, re.DOTALL) 
                
                   # print(filename)
                
                    # Obtem o cargo
                    expRegularCargo1 = x1.group()+"\s{0,}para\s{0,}(.*)\s{0,}cargo\s{0,}(.*)\s{0,}(.*)\s{0,}(.*)\s{0,}\."
                    cargo1=re.search(expRegularCargo1,aux)
                    cargoAux = ""
                                    
                    for i in range(2,len(cargo1.groups())):
                        cargoAux = cargoAux + cargo1.group(i)
                        print("G" + str(i) + " "+ cargo1.group(i))
                    
                    # grava arquivo csv
                    with open('C:\\temp\\es_gov\\'+filename+'_FINAL.csv', 'a', newline='') as csvfile:
                        writer = csv.writer(csvfile, delimiter=';')
                        lista = [0,x1,cargoAux,filename,"Estadual","ES"]
                        writer.writerow(lista)
                
                except Exception as e:
                    print(e)
                    continue

In [ ]:
# Faz merge dos cvs gerados Estadual - ES
import os

f = open('C:\\temp\\es_gov\\CONSOLIDADO.csv', 'a')
for file in os.listdir("C:\\temp\\es_gov"):
    if file.endswith(".csv") and not file.startswith("CONSOLIDADO") and not file.startswith("serra"):
        with open("C:\\temp\\es_gov\\"+file, 'r') as read_obj:
            for line in read_obj:
                f.write(line)
f.close()

In [ ]:
# testando a logica acima:

# FOI INCORPORADO ACIMA NO LOOP DE ARQUIVOS

import re

exemplo1 = " de acordo com o Art.12, inciso II, da Lei Complementar nº 46, de 31 de janeiro de 1994,  EMANUEL MALTA ROSA, para exercer o cargo de provimento  em comissão de Assessor  Jurídico do Sistema Penal, Ref. "

exemplo2 = ", de acordo com o Art.  12, II, da Lei Complementar 46/94 , JURACY DE ALMEIDA FILHO, para o cargo comissionado de  Chefe de CIRETRAN de Guaçuí/ES, Ref. DC-02.Vitória, 7 de dezembro de 2020"


# \, virgula
# \s{1,}   um ou mais espaços em branco
# \s{0,} igual a \s*   opcional, um ou mais espaços branco
# [A-Z]{3,}  3 ou mais maiusculas
# [a-zA-Z]*  uma string, com maiusculas e minusculas
# [A-Z*]  ter uma string em maiuscula ou nao  ( nem todos tem o 3o sobrenome)

#expRegular = "[0-9]\s{0,}\,\s{1,}[A-Z]{3,}\s[A-Z]{2,}\s[A-Z]{2,}\s*[A-Z]*\,"
expRegular = "\,\s{1,}[A-Z]{3,}\s[A-Z]{2,}\s[A-Z]{2,}\s*[A-Z]*\,"

x1 = re.search(expRegular, exemplo1) 
x2 = re.search(expRegular, exemplo2) 
print(x1)
print(x2)



In [ ]:
# Obtendo  os cargos 

# FOI INCORPORADO ACIMA NO LOOP DE ARQUIVOS

print(x1.group())
expRegularCargo1 = x1.group()+"\s{0,}para(.*?)o cargo(.*?)Ref."
cargo1=re.search(expRegularCargo1,exemplo1)
print(cargo1.group())


print(x2.group())
expRegularCargo2 = x2.group()+"\s{0,}para(.*?)o cargo(.*?)Ref."
cargo2=re.search(expRegularCargo2,exemplo2)
print(cargo2.group())